<a href="https://colab.research.google.com/github/margaretmz/CartoonGAN-e2e-tflite-tutorial/blob/master/ml/metadata/Add_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://github.com/margaretmz/selfie2anime-with-tflite/blob/master/ml/add-meta-data-Colab/Add%20metadata%20to%20selfie2anime.ipynb. 

TensorFlow Lite meatdata: https://www.tensorflow.org/lite/convert/metadata.

In [1]:
!pip install tflite-support

     |████████████████████████████████| 399kB 3.3MB/s 
  Using cached https://files.pythonhosted.org/packages/89/e3/d576f6f02bc75bacbc3d42494e8f1d063c95617d86648dba243c2cb3963e/pybind11-2.5.0-py2.py3-none-any.whl
  Created wheel for tflite-support: filename=tflite_support-0.1.0a1-cp36-cp36m-linux_x86_64.whl size=3950403 sha256=0c10fc5f20f1c78c9c9fe934c4639a83e041493a2a464b1f45b09adae2f3e4bb
  Stored in directory: /root/.cache/pip/wheels/5f/65/21/4fa08c598c3e79a1a5260646eb37204d4859fc3da0bfeb8011
Successfully built tflite-support


In [2]:
import os
import tensorflow as tf
from absl import flags

In [3]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

In [4]:
!mkdir model_without_metadata
!mkdir model_with_metadata

In [5]:
!wget https://storage.googleapis.com/cartoon_gan/fixed_shaped_models/whitebox_cartoon_gan_dr.tflite
!wget https://storage.googleapis.com/cartoon_gan/fixed_shaped_models/whitebox_cartoon_gan_fp16.tflite
!wget https://storage.googleapis.com/cartoon_gan/fixed_shaped_models/whitebox_cartoon_gan_int8.tflite

!mv *.tflite model_without_metadata/

--2020-08-03 04:59:31--  https://storage.googleapis.com/cartoon_gan/fixed_shaped_models/whitebox_cartoon_gan_dr.tflite
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2602000 (2.5M) [application/octet-stream]
Saving to: ‘whitebox_cartoon_gan_dr.tflite’

whitebox_cartoon_ga 100%[===================>]   2.48M  --.-KB/s    in 0.01s   

2020-08-03 04:59:31 (215 MB/s) - ‘whitebox_cartoon_gan_dr.tflite’ saved [2602000/2602000]

--2020-08-03 04:59:33--  https://storage.googleapis.com/cartoon_gan/fixed_shaped_models/whitebox_cartoon_gan_fp16.tflite
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, a

In [6]:
# This is where we will export a new .tflite model file with metadata, and a .json file with metadata info
EXPORT_DIR = "model_with_metadata"

In [7]:
class MetadataPopulatorForGANModel(object):
  """Populates the metadata for the CartoonGAN model."""

  def __init__(self, model_file):
    self.model_file = model_file
    self.metadata_buf = None

  def populate(self):
    """Creates metadata and then populates it for a style transfer model."""
    self._create_metadata()
    self._populate_metadata()
  
  def _create_metadata(self):
    """Creates the metadata for the CartoonGAN model."""

    # Creates model info.
    model_meta = _metadata_fb.ModelMetadataT()
    model_meta.name = "CartoonGAN" 
    model_meta.description = ("Cartoonizes an image. Reference: https://bit.ly/cartoon-gan. TFLiteConverter used from tf-nightly.")
    model_meta.version = "v1"
    model_meta.author = "TensorFlow"
    model_meta.license = ("Apache License. Version 2.0 "
                          "http://www.apache.org/licenses/LICENSE-2.0.")

    # Creates info for the input, normal image.
    input_image_meta = _metadata_fb.TensorMetadataT()
    input_image_meta.name = "source_image"
    # if self.model_type=="other":
    input_image_meta.description = (
            "The expected image is 520 x 520, with three channels "
            "(blue, red, and green) per pixel. Each value in the tensor is between"
            " -1 and 1.")
    # elif self.model_type=="fp16":
    #     input_image_meta.description = (
    #         "The expected image is 520 x 520, with three channels "
    #         "(red, blue, and green) per pixel. Each value in the tensor is between"
    #         " -1 and 1.")
    input_image_meta.content = _metadata_fb.ContentT()
    input_image_meta.content.contentProperties = (
        _metadata_fb.ImagePropertiesT())
    input_image_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    input_image_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    input_image_normalization = _metadata_fb.ProcessUnitT()
    input_image_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    input_image_normalization.options = _metadata_fb.NormalizationOptionsT()
    input_image_normalization.options.mean = [127.5]
    input_image_normalization.options.std = [127.5]
    input_image_meta.processUnits = [input_image_normalization]
    input_image_stats = _metadata_fb.StatsT()
    input_image_stats.max = [1.0]
    input_image_stats.min = [-1.0]
    input_image_meta.stats = input_image_stats


    # Creates output info, cartoonized image
    output_image_meta = _metadata_fb.TensorMetadataT()
    output_image_meta.name = "cartoonized_image"
    output_image_meta.description = "Image cartoonized."
    output_image_meta.content = _metadata_fb.ContentT()
    output_image_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
    output_image_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    output_image_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    output_image_normalization = _metadata_fb.ProcessUnitT()
    output_image_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    output_image_normalization.options = _metadata_fb.NormalizationOptionsT()
    output_image_normalization.options.mean = [0.0]
    output_image_normalization.options.std = [1.0]
    output_image_meta.processUnits = [output_image_normalization]
    output_image_stats = _metadata_fb.StatsT()
    output_image_stats.max = [255.0]
    output_image_stats.min = [0.0]
    output_image_meta.stats = output_image_stats

    # Creates subgraph info.
    subgraph = _metadata_fb.SubGraphMetadataT()
    subgraph.inputTensorMetadata = [input_image_meta] 
    subgraph.outputTensorMetadata = [output_image_meta] 
    model_meta.subgraphMetadata = [subgraph]

    b = flatbuffers.Builder(0)
    b.Finish(
        model_meta.Pack(b),
        _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
    self.metadata_buf = b.Output()

  def _populate_metadata(self):
    """Populates metadata to the model file."""
    populator = _metadata.MetadataPopulator.with_model_file(self.model_file)
    populator.load_metadata_buffer(self.metadata_buf)
    populator.populate()

In [10]:
def populate_metadata(model_file):
  """Populates the metadata using the populator specified.
  Args:
      model_file: valid path to the model file.
      model_type: a type defined in StyleTransferModelType .
  """

  # Populates metadata for the model.
  model_file_basename = os.path.basename(model_file)
  export_path = os.path.join(EXPORT_DIR, model_file_basename)
  tf.io.gfile.copy(model_file, export_path, overwrite=True)

  populator = MetadataPopulatorForGANModel(export_path) 
  populator.populate()

  # Displays the metadata that was just populated into the tflite model.
  displayer = _metadata.MetadataDisplayer.with_model_file(export_path)
  export_json_file = os.path.join(
      EXPORT_DIR,
      os.path.splitext(model_file_basename)[0] + ".json")
  json_file = displayer.get_metadata_json()
  with open(export_json_file, "w") as f:
    f.write(json_file)
  print("Finished populating metadata and associated file to the model:")
  print(export_path)
  print("The metadata json file has been saved to:")
  print(os.path.join(EXPORT_DIR,
                   os.path.splitext(model_file_basename)[0] + ".json"))

In [13]:
quantization = "fp16" #@param ["dr", "int8", "fp16"]
tflite_model_path = f"whitebox_cartoon_gan_{quantization}.tflite" 
MODEL_FILE = "/content/model_without_metadata/{}".format(tflite_model_path)
populate_metadata(MODEL_FILE)

Finished populating metadata and associated file to the model:
model_with_metadata/whitebox_cartoon_gan_fp16.tflite
The metadata json file has been saved to:
model_with_metadata/whitebox_cartoon_gan_fp16.json


In [14]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [15]:
!gsutil -m cp -r model_with_metadata/* gs://cartoon_gan/fixed_shaped_models/with_metadata/

Copying file://model_with_metadata/whitebox_cartoon_gan_dr.json [Content-Type=application/json]...
Copying file://model_with_metadata/whitebox_cartoon_gan_dr.tflite [Content-Type=application/octet-stream]...
Copying file://model_with_metadata/whitebox_cartoon_gan_int8.tflite [Content-Type=application/octet-stream]...
Copying file://model_with_metadata/whitebox_cartoon_gan_fp16.json [Content-Type=application/json]...
Copying file://model_with_metadata/whitebox_cartoon_gan_int8.json [Content-Type=application/json]...
Copying file://model_with_metadata/whitebox_cartoon_gan_fp16.tflite [Content-Type=application/octet-stream]...
/ [6/6 files][  7.6 MiB/  7.6 MiB] 100% Done                                    
Operation completed over 6 objects/7.6 MiB.                                      
